In [ ]:
import pickle

with open('/home/gk/Downloads/Telegram Desktop/pair_dists_reversed_2.pkl', 'rb') as f:
    my_dict = pickle.load(f)

In [ ]:
my_dict

In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from tqdm import tqdm

num_points = int(np.sqrt(len(my_dict) * 2))

# Convert the dictionary into a distance matrix
distance_matrix = np.zeros((num_points+20, num_points+20))
for k, v in tqdm(my_dict.items()):
    i, j = k
    distance_matrix[i, j] = v
    distance_matrix[j, i] = v



In [ ]:
import pandas as pd
df = pd.DataFrame(distance_matrix)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
# import plotly.express as px

tsne = TSNE(n_components=2, perplexity=100, n_iter=1000, random_state=7575)
tsne_results = tsne.fit_transform(df)

In [ ]:
from sklearn.mixture import GaussianMixture as GMM


from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
tqdm.pandas()

def SelBest(arr:list, X:int)->list:
    '''
    returns the set of X configurations with shorter distance
    '''
    dx=np.argsort(arr)[:X]
    return arr[dx]

n_clusters=np.arange(30, 60)
# n_clusters=np.arange(10, 11)
sils=[]
sils_err=[]
iterations=20
for n in tqdm(n_clusters):
    tmp_sil=[]
    for _ in range(iterations):
        gmm=GMM(n, n_init=2, random_state=7575, covariance_type='diag', init_params='kmeans').fit(tsne_results) 
        labels=gmm.predict(tsne_results)
        sil=metrics.silhouette_score(tsne_results, labels, metric='euclidean')
        tmp_sil.append(sil)
    val=np.mean(SelBest(np.array(tmp_sil), int(iterations/5)))
    err=np.std(tmp_sil)
    sils.append(val)
    sils_err.append(err)

plt.errorbar(n_clusters, sils, yerr=sils_err)
plt.title("Silhouette Scores", fontsize=20)
plt.xticks(n_clusters)
plt.xlabel("N. of clusters")
plt.ylabel("Score")

dct_tmp = {n_clusters[i]: sils[i] for i in range(len(n_clusters))}
max_score_clust = max(dct_tmp, key=dct_tmp.get)

In [ ]:
eps_range = np.arange(0.1, 2.1, 0.1)
min_samples_range = np.arange(2, 21, 1)
best_score = -1
best_eps = 0
best_min_samples = 0
for eps in tqdm(eps_range):
    for min_samples in min_samples_range:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(tsne_results)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        if 20 <= n_clusters <= 50:  # check if number of clusters is between 20 and 50
            score = silhouette_score(tsne_results, labels)
            if score > best_score:
                best_score = score
                best_eps = eps
                best_min_samples = min_samples

print(f"Best silhouette score: {best_score:.2f}, Best eps: {best_eps:.2f}, Best min samples: {best_min_samples}")

# decrease eps for smaller leafs
best_eps /= 2

# perform bdscan clustering on t-SNE output using the optimal parameters
dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples)
labels = dbscan.fit_predict(tsne_results)

In [ ]:
len(labels)

In [ ]:
import hdbscan

# find optimal parameters for hdbscan
min_cluster_size_range = np.arange(10, 30, 1)
best_score = -1
best_min_cluster_size = 0
for min_cluster_size in tqdm(min_cluster_size_range):
    hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size)
    hdbscan_labels = hdbscan_model.fit_predict(tsne_results)
    n_clusters = len(set(hdbscan_labels)) - (1 if -1 in hdbscan_labels else 0)
    if n_clusters >= 3:  # check if number of clusters is at least 3
        score = silhouette_score(tsne_results, hdbscan_labels)
        if score > best_score:
            best_score = score
            best_min_cluster_size = min_cluster_size

print(f"Best silhouette score: {best_score:.2f}, Best min_cluster_size: {best_min_cluster_size}")

# perform hdbscan clustering on t-SNE output using the optimal parameters
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=best_min_cluster_size)
hdbscan_labels = hdbscan_model.fit_predict(tsne_results)

In [ ]:
from sklearn.cluster import OPTICS

min_samples_range = np.arange(2, 21, 1)
best_score = -1
best_min_samples = 0
for min_samples in tqdm(min_samples_range):
    optics = OPTICS(min_samples=min_samples)
    optics.fit(tsne_results)
    labels = optics.labels_
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    if n_clusters >= 3:  # check if number of clusters is at least 3
        score = silhouette_score(tsne_results, labels)
        if score > best_score:
            best_score = score
            best_min_samples = min_samples

print(f"Best silhouette score: {best_score:.2f}, Best min samples: {best_min_samples}")

# perform optics clustering on t-SNE output using the optimal parameters
optics = OPTICS(min_samples=best_min_samples)
optics.fit(tsne_results)
labels = optics.labels_

In [ ]:
# max_score_clust

In [ ]:
# Compute the DBSCAN clustering with your custom distance matrix
# gmm = GMM(n_components=max_score_clust, random_state=7575, covariance_type='diag', init_params='kmeans').fit_predict(tsne_results)

# # Compute the silhouette score
# labels = dbscan.labels_
# silhouette_avg = silhouette_score(distance_matrix, labels, metric='precomputed')
# print("The silhouette score is:", silhouette_avg)

In [ ]:
import networkx as nx

G = nx.read_graphml('/home/gk/code/migration_project/G12.graphml')

import geopandas as gpd
from shapely.wkt import loads

# assuming you already have a graph G with node data that includes a "geometry" attribute
# use the nodes method from networkx to get a NodeDataView object
nodedata = G.nodes(data=True)

# create a list of dictionaries to store the attributes of each node
node_attrs = []
for n, data in nodedata:
    node_dict = data.copy()
    node_dict["nodeID"] = n
    node_dict["geometry"] = loads(node_dict["geometry"])
    node_attrs.append(node_dict)

# create a GeoDataFrame from the list of node attributes, using the "geometry" attribute as the geometry
nodes_gdf = gpd.GeoDataFrame(node_attrs, geometry='geometry', crs=4326)

In [ ]:
nodes_gdf.shape

In [ ]:
nodes_gdf.head()

In [ ]:
nodes_gdf['cluster'] = hdbscan_labels

In [ ]:
nodes_gdf.to_file('nodes_gdf_HDBSCAN_4.geojson')